# Time-Spacial Exploration

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make charts from `repteis` database with focus on time and space exploration. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.m
    

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
# alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig', low_memory=False)

<br>

<font size=5>**Color palette per Order**</font>

The image below served as inspiration (https://color.adobe.com/create/image)

<img src="./src/paleta_cores.jpeg" width='500px'>

Cores: 

- verde_escuro: #284021
- verde_claro: #88BF11
- amarelo: #D9CB0B
- laranja: #D99311
- laranja_escuro: #BF4417
- marrom-_laro: #BF8D7A

In [3]:
# p.s.: Caudata is an error and should be removed
cores_ordem = {
    'Squamata': '#BF4417',
    'Testudines': '#D9CB0B', 
    'Crocodylia': '#284021'
}

ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

<br>

## Graph: Exploring time-spacial evolution of the database

In [8]:
# corrects a typo (Améica do Sul)
NewTable['continente'] = NewTable['continente'].apply(lambda x: 'América do Sul' if x=='Améica do Sul' else x)

In [9]:
# looking good...
NewTable['continente'].value_counts()

América do Sul      21927
América Central       135
América do Norte      103
Ásia                   54
#N/D                   25
África                 13
Europa                  9
Name: continente, dtype: int64

### Chart 1: Evolution per time and continent

In [10]:
# agrupando por Ano e Continente
teste = NewTable.groupby(['ano_coleta','continente', 'ordem']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [11]:
g1 = alt.Chart(teste, title='Evolução temporal da coleção por continente').mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('continente', title='Continente', 
             sort=alt.EncodingSortField('counts', op="count", order='descending')), 
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['continente','ano_coleta','ordem','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_continente.html')

# g1

<br>

### Chart 2: Evolution per time and country

In [54]:
# agrupando por Ano e Continente
teste1 = NewTable.groupby(['ano_coleta','pais', 'ordem']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [55]:
# retirei Brasil (será detalhado por Estados depois)
g1 = alt.Chart(teste1[teste1['pais'] != 'Brasil'], title='Animais coletados por ano em outros países',
              width=800).mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('pais', title='País', 
             sort=alt.EncodingSortField('counts', op="sum", order='descending')), 
    size=alt.Size('counts', scale= alt.Scale(range=[20,250])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['pais','ano_coleta','ordem','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_pais.html')

# g1

<br>

### Chart 3: Evolution per time and brazilian states

In [128]:
# agrupando por Ano e Continente
teste2 = NewTable[NewTable['pais'] == 'Brasil']
teste2 = teste2.groupby(['ano_coleta','estado_ou_provincia', 'ordem']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

#### creating column with brazilian regions

In [129]:
regioes = {
    'Rio de Janeiro':'SE',
    'São Paulo':'SE',
    'Espírito Santo': 'SE',
    'Pernambuco':'NE',
    'Santa Catarina':'S',
    'Amazonas':'N',
    'Goiás':'CO',
    'Roraima':'N',
    'Pará':'N',
    'Mato Grosso':'CO',
    'Acre': 'N',
    'Bahia': 'NE',
    'Minas Gerais': 'SE',
    'Mato Grosso do Sul': 'CO',
    'Paraná': 'S',
    'Rondônia': 'N',
    'Ceará': 'NE',
    'Maranhão': 'N',
    'Rio Grande do Sul': 'S',
    'Paraíba': 'NE',
    'Distrito Federal': 'CO',
    'Alagoas': 'NE',
    'Amapá':'N',
    'Piauí': 'NE',
    'Brasília': 'CO',
    'Tocantins': 'N',
    'Rio Grande do Norte': 'NE',
    'Sergipe': 'NE',
    'Minas Gerais/Goiás/Distrito Federal': 'CO',
    'Santa Catarina-Rio Grande do Sul': 'S'
}

# criando coluna com as regiões
teste2['regiao'] = teste2['estado_ou_provincia'].apply(lambda x: regioes[str(x)])

# coluna com estado, regiao
teste2['regiao_e_estado'] = teste2['estado_ou_provincia'] + ', ' + teste2['regiao']

# ordenando por região e soma das contagens
sorting = teste2.groupby(['regiao_e_estado', 'regiao']).sum()['counts'].reset_index(
                                                                ).rename(columns={'counts':'soma'})
sorting = sorting.sort_values(['regiao','soma'], ascending=False)['regiao_e_estado'].unique()

In [130]:
g1 = alt.Chart(teste2, title='Animais coletados por ano nos Estados Brasileiros').mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','ordem','counts'])
)

# OBS: marca o background da região que eu quiser, mas distorce as cores
# rule = alt.Chart(teste2[teste2['regiao'].isin(['SE','NE','CO'])], 
#                  title='Animais coletados por ano nos Estados Brasileiros').mark_rule(
#                     opacity=0.1, size=19, color='#dedede', filled=False, fill='green').encode(
#     y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
#             sort= sorting, axis=alt.Axis(labels=True))
# )

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros.html')

# g1

<br>
<br>

<font color='red' size='5'> Separando por períodos </font>

In [131]:
# periodos: <1938, >=1938 e <= 1985, > 1985 
ano_corte1 = 1938
ano_corte2 = 1985

periodo1 = teste2[teste2['ano_coleta'] < 1938].copy()
periodo2 = teste2[(teste2['ano_coleta'] >= 1938) & (teste2['ano_coleta'] <= 1985)].copy()
periodo3 = teste2[teste2['ano_coleta'] > 1985].copy()

### periodo 1: até 1937

In [132]:
g1 = alt.Chart(periodo1, title='Animais coletados por ano nos Estados Brasileiros (até 1937)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','ordem','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-ate1937.html')

# g1

### periodo 2: de 1938 a 1985

In [133]:
g1 = alt.Chart(periodo2, title='Animais coletados por ano nos Estados Brasileiros (de 1938 a 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','ordem','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-de1938a1985.html')

# g1

### periodo 3: após 1985

In [134]:
g1 = alt.Chart(periodo3, title='Animais coletados por ano nos Estados Brasileiros (após 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','ordem','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-apos1985.html')

# g1

<br>

## Lat Long

In [16]:
def regiao(est):
    if est not in regioes.keys():
        return np.NAN
    else:
        return regioes[est]
    
NewTable['regiao'] = NewTable['estado_ou_provincia'].apply(regiao)

In [17]:
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

world = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1')

# world

In [18]:
# dynamic version
slider = alt.binding_range(min=1900, max=2016, step=1)
select_year = alt.selection_single(name=' ', fields=['ano_coleta'],
                                   bind=slider, init={'ano_coleta': 1935})

# data = NewTable[(NewTable['long'] < 10) & (NewTable['long'] >-150)].copy()
data = NewTable.copy()

back = alt.Chart(data).mark_circle(opacity=0.7, color='lightgray').encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem'])
)

front = alt.Chart(data).mark_circle().encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color = alt.Color('ordem:N', title='Ordem', scale=
                     alt.Scale(range=cores, domain=ordens)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem'])
).add_selection(
    select_year
).transform_filter(
    select_year
)

equador = alt.Chart(data[data['lat'] == 0], width=800, height= 400).mark_rule(color='black').encode(
    y = alt.Y('lat:Q', axis=alt.Axis(grid=False))
)


g = alt.layer(world, back, front).properties(
    width= 800,
    height= 500
)  # tirei o equador

# ajustando tamanho de fonte
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/time-spacial/lat_long-worldwide.html')

# g

<br>

<font color='red' size='5'>Lat long (layout 2 estático)</font>

In [49]:
# https://nextjournal.com/sdanisch/cartographic-visualization

# seletores
select_order = alt.selection_multi(fields=['ordem'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

tipos = NewTable['type_status'].unique()  # lista temporária! ajustar depois!

# AJUSTAR CORES

from vega_datasets import data

front = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color = alt.Color('ordem:N', title='Ordem', scale=
                     alt.Scale(range=cores, domain=ordens)),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','familia','ordem', 'type_status'])
).add_selection(select_order, select_type).transform_filter(select_order).transform_filter(select_type)

mapa = alt.layer(
     # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#f2f2f2'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(data.world_110m.url, 'countries')).mark_geoshape(
        fill='#white', stroke='black', strokeWidth=0.5
    ),
    front
).properties(
    width=800,
    height=500
)

mapa = mapa.project(
    type='naturalEarth1', scale=140
).configure_view(stroke=None)

mapa = mapa.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# mapa.save('./graphs/time-spacial/lat_long-worldwide-static-grid.html')

# mapa

In [52]:
teste = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color = alt.Color('ordem:N', title='Ordem', scale = alt.Scale(range=cores, domain=ordens)),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','familia','ordem', 'type_status'])
).project(type='naturalEarth1').add_selection(select_order, 
                            select_type).transform_filter(select_order).transform_filter(select_type)

temp = (world + teste).properties(width=800, height=500).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-worldwide-static.html')
# temp

<br>

### Latin America

In [53]:
# seletores
select_order = alt.selection_multi(fields=['ordem'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

tipos = NewTable['type_status'].unique()  # lista temporária! ajustar depois!

# mapa
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

latinAmerica = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1', translate=[680, 150], scale=450)

# pontos
teste = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color = alt.Color('ordem:N', title='Order', scale = alt.Scale(range=cores, domain=ordens)),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','familia','ordem', 'type_status'])
).project(type='naturalEarth1', translate=[680, 150], scale=450).add_selection(select_order,
                                select_type).transform_filter(select_order).transform_filter(select_type)


# configurando grafico
temp = (latinAmerica + teste).properties(width=600, height=600).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-latinAmerica-static.html')
# temp

**Fim!**